# Preparing Victron data for dashboards

Based on the [Victron API](https://docs.victronenergy.com/vrmapi/overview.html)

***include in documentation the things you only have to do once***

In [ ]:
import support_functions as s
import pandas as pd
import sqlite3

In [ ]:
db_path = "C:/Users/reeep/OneDrive/Desktop/e2_cleanup/esp_db/victron_db.db"
conn = sqlite3.connect(db_path)

In [ ]:
# TO SET UP THE DB (ONLY ONCE)
#setup_db(conn)

In [ ]:
creds = s.get_token('dlswee@gmail.com','')

In [ ]:
dates = pd.date_range('2022-08-01','2022-11-01',4)

In [ ]:
# get statis table for minigrid name and location

sites = s.get_basics(creds['token'],creds['user_id'])

site_info = []

for site in sites:
    site_dict = {}
    site_dict['name'] = site['name'] 
    site_dict['id'] = site['idSite'] 
    for val in site['extended']:
        if val['description'] in ['Latitude','Longitude']:
            site_dict[val['description']] = val['rawValue']

    site_info.append(site_dict)
    
df_mg_data = pd.DataFrame(site_info)
df_mg_data.to_csv('minigrid_location.csv', index=False)

In [ ]:
# Get metrics by site and date range

#in epoch format, make sure they are both on the top of the hour
#NOTE: you can only pull up to 30 days at oncedates = pd.date_range('2022-06-01','2022-10-01',5)

for site in sites:
    for i in range(len(dates)-1):
        s.get_raw(creds['token'], 
                  site['idSite'], 
                  conn, 
                  int(dates[i].asm8.astype('int64')/1000000000), 
                  int(dates[i+1].asm8.astype('int64')/1000000000))
        print(site['name'] + ' : ' +str(site['idSite']) + ' : ' + str(dates[i+1]))

In [ ]:
# Update main metrics table, including de-duplicating records
s.update_ssot(conn)

In [ ]:
df_ssot = pd.read_sql('select * from ssot', conn, parse_dates=['reading_time'])

In [ ]:
conn.close()

In [ ]:
df_ssot.to_csv('C:/Users/reeep/OneDrive/Desktop/minigrid_readings_20221016.csv', index=False)